## Imports, data load, metric function definition

In [218]:
import numpy as np
import pandas as pd
from sklearn.neighbors import BallTree

In [219]:
X_train = np.load('X_train_surge.npz')
Y_train = pd.read_csv('Y_train_surge.csv')
X_test = np.load('X_test_surge.npz')

In [220]:
def split_train_set(X, Y, val_size=0.2, seed=None):
    rng = np.random.default_rng(seed)
    
    nb_examples = len(Y)
    val_examples = int(val_size * nb_examples)
    
    val_indices = rng.choice(nb_examples, size=val_examples, replace=False)
    
    train_indices = np.setdiff1d(
        np.arange(nb_examples),
        val_indices,
        assume_unique=True
    )
    train_indices = rng.permutation(train_indices)
    
    X_train = {}
    X_val = {}
    for feat in X.files:
        X_train[feat] = X[feat][train_indices]
        X_val[feat] = X[feat][val_indices]
    
    return X_train, Y.iloc[train_indices], \
            X_val, Y.iloc[val_indices]

X_train, Y_train, X_val, Y_val = split_train_set(X_train, Y_train, val_size=0.091, seed=1234)

In [221]:
t_slp = X_train['t_slp'] / 3600
t_slp_delta = t_slp - t_slp[:, 0].reshape(-1, 1)
np.allclose(np.round(t_slp_delta), np.round(t_slp_delta)[0])

True

In [222]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

In [223]:
SLP_HEIGHT = SLP_WIDTH = 41
SLP_PER_EX = 40
T_SURGE_NORMALISATION = 240.
SLP_REL_TIMESTAMPS = np.arange(24*5, step=3) / T_SURGE_NORMALISATION


def normalised_tensor(array, mean, std):
        return torch.from_numpy((array - mean) / std)

def preprocessing(X, slp_mean=None, slp_std=None, t_slp_mean=None, t_slp_std=None,
                 surge_mean=None, surge_std=None):
    slp = X['slp'].reshape(-1, SLP_PER_EX, SLP_HEIGHT, SLP_WIDTH)
    slp = np.roll(slp, shift=-11, axis=3)
    if slp_mean is None:
        slp_mean = np.mean(slp)
    if slp_std is None:
        slp_std = np.std(slp)
    slp = normalised_tensor(slp, slp_mean, slp_std)
    
    fst_slp = X['t_slp'][:, 0] / 3600
    fst_slp_tmp = fst_slp.reshape(-1, 1)
    
    def rel_surge_time(index):
        t_surge = X[index] / 3600
        t_surge -= fst_slp_tmp
        return torch.from_numpy(t_surge / T_SURGE_NORMALISATION)

    t_surge1_in = rel_surge_time('t_surge1_input')
    t_surge2_in = rel_surge_time('t_surge2_input')
    t_surge1_out = rel_surge_time('t_surge1_output')
    t_surge2_out = rel_surge_time('t_surge2_output')
    
    if t_slp_mean is None:
        t_slp_mean = np.mean(fst_slp)
    if t_slp_std is None:
        t_slp_std = np.std(fst_slp)
    fst_slp = normalised_tensor(fst_slp, t_slp_mean, t_slp_std)
    
    surge1 = X['surge1_input']
    surge2 = X['surge2_input']
    if surge_mean is None or surge_std is None:
        surges = np.concatenate([surge1, surge2], axis=None)
        surge_mean = np.mean(surges)
        surge_std = np.std(surges)
    surge1_in = normalised_tensor(surge1, surge_mean, surge_std)
    surge2_in = normalised_tensor(surge2, surge_mean, surge_std)
    
    return X['id_sequence'], slp, slp_mean, slp_std, \
            fst_slp, t_slp_mean, t_slp_std, \
            t_surge1_in, t_surge2_in, t_surge1_out, t_surge2_out, \
            surge1_in, surge2_in, surge_mean, surge_std

In [224]:
train_id_seq, train_slp, slp_mean, slp_std, \
train_fst_slp, t_slp_mean, t_slp_std, \
train_t_surge1_in, train_t_surge2_in, train_t_surge1_out, train_t_surge2_out, \
train_surge1_in, train_surge2_in, surge_mean, surge_std = preprocessing(X_train)

In [225]:
val_id_seq, val_slp, _, _, \
val_fst_slp, _, _, \
val_t_surge1_in, val_t_surge2_in, val_t_surge1_out, val_t_surge2_out, \
val_surge1_in, val_surge2_in, _, _ = preprocessing(X_val, slp_mean, slp_std, t_slp_mean, t_slp_std, surge_mean, surge_std)

In [226]:
test_id_seq, test_slp, _, _, \
test_fst_slp, _, _, \
test_t_surge1_in, test_t_surge2_in, test_t_surge1_out, test_t_surge2_out, \
test_surge1_in, test_surge2_in, _, _ = preprocessing(X_test, slp_mean, slp_std, t_slp_mean, t_slp_std, surge_mean, surge_std)

In [229]:
train_Y_id_seq = Y_train['id_sequence'].values
assert np.alltrue(train_Y_id_seq == train_id_seq), 'Data/label index mismatch'

train_surge1_out = Y_train.iloc[:, 1:11].values
train_surge1_out = normalised_tensor(train_surge1_out, surge_mean, surge_std)
train_surge2_out = Y_train.iloc[:, 11:].values
train_surge2_out = normalised_tensor(train_surge2_out, surge_mean, surge_std)

In [230]:
val_Y_id_seq = Y_val['id_sequence'].values
assert np.alltrue(val_Y_id_seq == val_id_seq), 'Data/label index mismatch'

val_surge1_out = Y_val.iloc[:, 1:11].values
val_surge1_out = normalised_tensor(val_surge1_out, surge_mean, surge_std)
val_surge2_out = Y_val.iloc[:, 11:].values
val_surge2_out = normalised_tensor(val_surge2_out, surge_mean, surge_std)

In [253]:
from torch.utils.data import TensorDataset, DataLoader

resnet_versions = {
    18 : models.resnet18,
    34 : models.resnet34,
    50 : models.resnet50,
}

batch_size = 64

train_dataset = TensorDataset(
    train_slp,
#     train_fst_slp,
    train_t_surge1_in,
    train_surge1_in,
    train_t_surge2_in,
    train_surge2_in,
    train_t_surge1_out,
    train_surge1_out,
    train_t_surge2_out,
    train_surge2_out,
)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

val_dataset = TensorDataset(
    val_slp,
#     val_fst_slp,
    val_t_surge1_in,
    val_surge1_in,
    val_t_surge2_in,
    val_surge2_in,
    val_t_surge1_out,
    val_surge1_out,
    val_t_surge2_out,
    val_surge2_out,
)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

test_dataset = TensorDataset(
    test_slp,
#     test_fst_slp,
    test_t_surge1_in,
    test_surge1_in,
    test_t_surge2_in,
    test_surge2_in,
    test_t_surge1_out,
    test_t_surge2_out
)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

class Conv2d(nn.Conv2d):
    """Weight standardisation
    (see https://arxiv.org/pdf/1903.10520.pdf and https://github.com/joe-siyuan-qiao/WeightStandardization)
    """

    def __init__(self, in_channels, out_channels, kernel_size, stride=1,
                 padding=0, dilation=1, groups=1, bias=True):
        super(Conv2d, self).__init__(in_channels, out_channels, kernel_size, stride,
                 padding, dilation, groups, bias)

    def forward(self, x):
        weight = self.weight
        weight_mean = weight.mean(dim=1, keepdim=True).mean(dim=2,
                                  keepdim=True).mean(dim=3, keepdim=True)
        weight = weight - weight_mean
        std = weight.view(weight.size(0), -1).std(dim=1).view(-1, 1, 1, 1) + 1e-5
        weight = weight / std.expand_as(weight)
        return F.conv2d(x, weight, self.bias, self.stride,
                        self.padding, self.dilation, self.groups)
    
class Network(nn.Module):
    
    def __init__(self, resnet_layers, out_features):
        super().__init__()
        self.resnet = resnet_versions[resnet_layers]()
        self.resnet.conv1 = nn.Conv2d(SLP_PER_EX, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        
        # Change the number of out_features from 1000 to `out_features`
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, out_features)
        self.slp_embed_size = out_features
        self.fc1 = nn.Linear(20, 1)
        self.fc2 = nn.Linear(20, 1)
        self.lstm = nn.LSTM(input_size=1, hidden_size=out_features, num_layers=1, bias=True, batch_first=True, \
                            dropout=.2, bidirectional=False, proj_size=1)
        # LSTM:
        # Add prediction time?
        # Learn to associate pressure_mapt to their timestamps
        
    def input_embedding(self, t_surge_in, surge_in):
        surge = torch.cat([t_surge_in, surge_in], dim=1)
        surge = torch.relu(self.fc1(surge))
        surge = self.fc2(surge)
        return surge
        
    def time_embedding(self, t_surge_out):
        t_surge_out = torch.relu(self.fc3(t_surge_out))
        t_surge_out = self.fc4(t_surge_out)
        return t_surge_out
        
    def forward(self, slp, t_surge1_in, surge1_in, t_surge2_in, surge2_in, t_surge1_out, t_surge2_out):
        cell = self.resnet(slp)
        c_0 = torch.cat([cell, cell], dim=0).unsqueeze(0)
        
        surge1 = torch.cat([t_surge1_in, surge1_in], dim=1)
        hidden1 = self.fc1(surge1)
        surge2 = torch.cat([t_surge2_in, surge2_in], dim=1)
        hidden2 = self.fc2(surge2)
        h_0 = torch.cat([hidden1, hidden2], dim=0).unsqueeze(0)
        
        lstm_input = torch.cat([t_surge1_out, t_surge2_out], dim=0).unsqueeze(2)
#         print(t_surge1_out.dtype, t_surge2_out.dtype, h_0.dtype, c_0.dtype)
        output, _ = self.lstm(lstm_input, (h_0, c_0))
        output = output.squeeze()
        return output[:len(slp)], output[len(slp):]

In [254]:
loss_weights = torch.linspace(1, 0.1, 10, requires_grad=False).to(device)
    
def surge_prediction_metric(surge1_true, surge2_true, surge1_pred, surge2_pred):
    surge1_score = torch.mean(torch.square(surge1_true - surge1_pred) * loss_weights)
    surge2_score = torch.mean(torch.square(surge2_true - surge2_pred) * loss_weights)
    return surge1_score + surge2_score

In [255]:
def train(model, loss_fn, optmiser, epochs):
    train_loss = []
    train_acc = []
    
    for epoch_num in range(epochs):
        model.train()
        running_loss = []

        for batch, (slp, t_surge1_in, surge1_in, t_surge2_in, surge2_in, t_surge1_out, surge1_out, t_surge2_out, surge2_out) in enumerate(train_dataloader):
            slp = slp.to(device)
            t_surge1_in = t_surge1_in.to(device)
            surge1_in = surge1_in.to(device)
            t_surge2_in = t_surge2_in.to(device)
            surge2_in = surge2_in.to(device)
            t_surge1_out = t_surge1_out.to(device)
            surge1_out = surge1_out.to(device)
            t_surge2_out = t_surge2_out.to(device)
            surge2_out = surge2_out.to(device)
            surge1_out_pred, surge2_out_pred = model(slp, t_surge1_in, surge1_in, t_surge2_in, surge2_in, t_surge1_out, t_surge2_out)
            
            loss = loss_fn(surge1_out, surge2_out, surge1_out_pred, surge2_out_pred)
            running_loss.append(loss.item())
            
            optimiser.zero_grad()
            loss.backward()
            optimiser.step()
            
        epoch_loss = np.mean(running_loss)
        train_loss.append(epoch_loss)
        print(f'Epoch {epoch_num+1:03d} | Loss: {epoch_loss:.4f}')
        
        if epoch_num % 5 == 0:
            val_loss = evaluate(model, loss_fn)
            print(f'\tVal loss: {val_loss:.4f}')
        
    return train_loss

def evaluate(model, loss_fn):
    with torch.no_grad():
        model.eval()
        losses = []
        for slp, t_surge1_in, surge1_in, t_surge2_in, surge2_in, t_surge1_out, surge1_out, t_surge2_out, surge2_out in val_dataloader:
            slp = slp.to(device)
            t_surge1_in = t_surge1_in.to(device)
            surge1_in = surge1_in.to(device)
            t_surge2_in = t_surge2_in.to(device)
            surge2_in = surge2_in.to(device)
            t_surge1_out = t_surge1_out.to(device)
            surge1_out = surge1_out.to(device)
            t_surge2_out = t_surge2_out.to(device)
            surge2_out = surge2_out.to(device)
            surge1_out_pred, surge2_out_pred = model(slp, t_surge1_in, surge1_in, t_surge2_in, surge2_in, t_surge1_out, t_surge2_out)
            
            loss = loss_fn(surge1_out, surge2_out, surge1_out_pred, surge2_out_pred)
            losses.append(loss.item())
        return np.mean(losses)

In [256]:
model = Network(18, 100).to(device)
optimiser = optim.Adam(model.parameters())
_ = train(model, surge_prediction_metric, optimiser, epochs=100)

/home/angelo/anaconda3/envs/dl/lib/python3.9/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch 001 | Loss: 0.9978
	Val loss: 0.8991
Epoch 002 | Loss: 0.8588
Epoch 003 | Loss: 0.6985
Epoch 004 | Loss: 0.6546
Epoch 005 | Loss: 0.6288
Epoch 006 | Loss: 0.6086
	Val loss: 0.5826
Epoch 007 | Loss: 0.5897
Epoch 008 | Loss: 0.5701
Epoch 009 | Loss: 0.5499
Epoch 010 | Loss: 0.5341
Epoch 011 | Loss: 0.5206
	Val loss: 0.5395
Epoch 012 | Loss: 0.5090
Epoch 013 | Loss: 0.4914
Epoch 014 | Loss: 0.4784
Epoch 015 | Loss: 0.4724
Epoch 016 | Loss: 0.4672
	Val loss: 0.5381
Epoch 017 | Loss: 0.4623
Epoch 018 | Loss: 0.4613
Epoch 019 | Loss: 0.4568
Epoch 020 | Loss: 0.4430
Epoch 021 | Loss: 0.4291
	Val loss: 0.6112
Epoch 022 | Loss: 0.4239
Epoch 023 | Loss: 0.4131
Epoch 024 | Loss: 0.3981
Epoch 025 | Loss: 0.3832
Epoch 026 | Loss: 0.3788
	Val loss: 0.5530
Epoch 027 | Loss: 0.3813
Epoch 028 | Loss: 0.3834
Epoch 029 | Loss: 0.3817
Epoch 030 | Loss: 0.3791
Epoch 031 | Loss: 0.3780
	Val loss: 0.5594
Epoch 032 | Loss: 0.3795
Epoch 033 | Loss: 0.3690
Epoch 034 | Loss: 0.3496
Epoch 035 | Loss: 0.3297

In [267]:
def write_submission(model, fn='submission.csv'):
    COLUMNS = [
        'surge1_t0', 'surge1_t1', 'surge1_t2', 'surge1_t3', 'surge1_t4',
        'surge1_t5', 'surge1_t6', 'surge1_t7', 'surge1_t8', 'surge1_t9',
        'surge2_t0', 'surge2_t1', 'surge2_t2', 'surge2_t3', 'surge2_t4',
        'surge2_t5', 'surge2_t6', 'surge2_t7', 'surge2_t8', 'surge2_t9' ]
    surge1_output = []
    surge2_output = []

    with torch.no_grad():
        for slp, t_surge1_in, surge1_in, t_surge2_in, surge2_in, t_surge1_out, t_surge2_out in test_dataloader:
            slp = slp.to(device)
            t_surge1_in = t_surge1_in.to(device)
            surge1_in = surge1_in.to(device)
            t_surge2_in = t_surge2_in.to(device)
            surge2_in = surge2_in.to(device)
            t_surge1_out = t_surge1_out.to(device)
            t_surge2_out = t_surge2_out.to(device)
            surge1_out_pred, surge2_out_pred = model(slp, t_surge1_in, surge1_in, t_surge2_in, surge2_in, t_surge1_out, t_surge2_out)
            surge1_output.append(surge1_out_pred)
            surge2_output.append(surge2_out_pred)

    surge1_output = torch.cat(surge1_output, dim=0)
    surge2_output = torch.cat(surge2_output, dim=0)
    test_outputs = torch.cat([surge1_output, surge2_output], dim=1).detach().cpu().numpy()

    test_df = pd.DataFrame(test_outputs, index=test_id_seq, columns=COLUMNS)
    test_df.index.name = 'id_sequence'
    test_df.to_csv(fn)
    return test_df

In [268]:
df = write_submission(model)

In [266]:
df

,surge1_t0,surge1_t1,surge1_t2,surge1_t3,surge1_t4,surge1_t5,surge1_t6,surge1_t7,surge1_t8,surge1_t9,surge2_t0,surge2_t1,surge2_t2,surge2_t3,surge2_t4,surge2_t5,surge2_t6,surge2_t7,surge2_t8,surge2_t9
id_sequence,,,,,,,,,,,,,,,,,,,,
5600,-1.522672,-1.640056,-1.812372,-1.873276,-1.911106,-1.892778,-1.814209,-1.698588,-1.570127,-1.445986,-1.077388,-1.206931,-1.322765,-1.355515,-1.416860,-1.485878,-1.503089,-1.474643,-1.412876,-1.315216
5601,-0.172018,-0.244204,-0.333128,-0.247466,0.008339,0.311038,0.669055,1.009192,1.198229,1.189028,0.016760,-0.052519,-0.104194,-0.077168,0.124576,0.411364,0.781786,1.110761,1.277642,1.251851
5602,-0.728413,-1.103798,-1.081307,-0.869735,-0.684275,-0.531688,-0.410095,-0.311700,-0.223303,-0.145472,1.353438,1.628443,1.170808,0.442155,0.039545,-0.142167,-0.216019,-0.203616,-0.161684,-0.113963
5603,-0.214031,0.022230,0.161170,0.047790,-0.173768,-0.312626,-0.312222,-0.243562,-0.164647,-0.095614,1.032095,1.588513,1.518492,0.934075,0.517910,0.386632,0.364263,0.341955,0.313909,0.286394
5604,0.188806,0.256014,0.136325,-0.055788,-0.255821,-0.438868,-0.531611,-0.504565,-0.409072,-0.300028,-0.038583,0.077969,-0.076706,-0.310813,-0.561307,-0.757425,-0.837745,-0.798326,-0.687479,-0.554552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6104,-0.202868,-0.458387,-0.676621,-0.850127,-0.801504,-0.661781,-0.550004,-0.469796,-0.375337,-0.267382,0.679106,-0.081019,-0.426764,-0.630051,-0.645582,-0.500996,-0.302407,-0.138851,-0.036838,0.022590
6105,0.004178,-0.027268,-0.104875,-0.129565,-0.042405,0.124772,0.274499,0.309280,0.270955,0.226377,0.155758,-0.343349,-0.494503,-0.406895,-0.211891,0.009162,0.194007,0.265217,0.251194,0.218806
6106,-0.429599,-0.720932,-0.728523,-0.629140,-0.443044,-0.221718,0.056978,0.360464,0.566990,0.590129,0.005821,-0.308064,-0.262538,-0.246235,-0.120654,0.120976,0.452356,0.784378,0.954932,0.906470


## Benchmark
Train using kNN of pressure fields at two instants in time, with 40 neighbours

In [4]:
def surge_prediction_metric(dataframe_y_true, dataframe_y_pred):
    weights = np.linspace(1, 0.1, 10)[np.newaxis]
    surge1_columns = [
        'surge1_t0', 'surge1_t1', 'surge1_t2', 'surge1_t3', 'surge1_t4',
        'surge1_t5', 'surge1_t6', 'surge1_t7', 'surge1_t8', 'surge1_t9' ]
    surge2_columns = [
        'surge2_t0', 'surge2_t1', 'surge2_t2', 'surge2_t3', 'surge2_t4',
        'surge2_t5', 'surge2_t6', 'surge2_t7', 'surge2_t8', 'surge2_t9' ]
    surge1_score = (weights * (dataframe_y_true[surge1_columns].values - dataframe_y_pred[surge1_columns].values)**2).mean()
    surge2_score = (weights * (dataframe_y_true[surge2_columns].values - dataframe_y_pred[surge2_columns].values)**2).mean()

    return surge1_score + surge2_score

In [5]:
nfields = 2; time_step_slp = 8
slp_train = []
slp_all = X_train['slp']
for i in range(5559):
    slp_train.append(np.ndarray.flatten(slp_all[i,-1]))
    for j in range(1,nfields):
        slp_train[-1] = np.concatenate( ( slp_train[-1], np.ndarray.flatten(slp_all[i,-1-j*time_step_slp]) ) )
slp_train = np.array(slp_train)

In [6]:
slp_test = []
slp_all_test = X_test['slp']
for i in range(509):
    slp_test.append(np.ndarray.flatten(slp_all_test[i,-1]))
    for j in range(1,nfields):
        slp_test[-1] = np.concatenate( ( slp_test[-1], np.ndarray.flatten(slp_all_test[i,-1-j*time_step_slp]) ) )
slp_test = np.array(slp_test)

In [7]:
tree = BallTree(slp_train)

In [8]:
surge_test_benchmark = []; k = 40
for i in range(509):
    dist, ind = tree.query([slp_test[i]], k=k)
    surge_test_benchmark.append(np.mean(surge_train[ind[0]], axis=0))
surge_test_benchmark = np.array(surge_test_benchmark)

In [9]:
y_columns = [f'surge1_t{i}' for i in range(10)] + [f'surge2_t{i}' for i in range(10)]
y_test_benchmark = pd.DataFrame(data=surge_test_benchmark, columns=y_columns, index=X_test['id_sequence'])
y_test_benchmark.to_csv('Y_test_benchmark.csv', index_label='id_sequence', sep=',')